# Sharp module

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот модуль нацелен на получение снимков более высокого разрешения, используя как снимки низкого разрешения, так и другие снимки высокого разрешения. Например, возьмем спутниковую систему [Landsat-8](https://landsat.usgs.gov/landsat-8). Каждый снимок содержит 12 каналов. Используя **PAN**-канал, разрешение которого составляет 15 м и, например, какналы **'B2'**, **'B3'**, **'B4'**, разрешение которых равно 30 м, можно получить те же три канала с разрешением 15 м.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Сам модуль включает в себя одну функцию, доступную для внешнего вызова:

- [pansharpening](#pansharpening "pansharpening");

А также ряд вспомогательных функций:

- [sizeKernel](#sizeKernel "sizeKernel");
- [psf](#psf "psf");
- [convolution](#convolution "convolution");
- [downscaling](#downscaling " downscaling");
- [getLinRegression](#getLinRegression "getLinRegression");
- [trend](#trend "trend");
- [residual](#residual "residual");
- [distance](#distance "distance");
- [CtoCsemivariogram](#CtoCsemivariogram "CtoCsemivariogram");
- [FtoCsemivariogram](#FtoCsemivariogram "FtoCsemivariogram");
- [krigingMatrix](#krigingMatrix "krigingMatrix");
- [krigingVector](#krigingVector "krigingVector");
- [getWeights](#getWeights "getWeights");

Ознакомиться с ними можно в [приложении](#Functions "Functions") или внутри [кода](#Script "Script").

## Sharpening

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Для реализации поставленной задачи применим подход [Area-to-point regression kriging](https://www.sciencedirect.com/science/article/pii/S0924271616000496) (ATPRK).

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Пусть имеется некоторое значение $Z^l_C(\mathbf{x_i})$ пикселя **C**, центр которого находится в точке $\mathbf{x_i}$ (канал **l**) и значение $Z_F(\mathbf{x_j})$ пикселя **F**, центр которого находится в точке $\mathbf{x_j}$ (канал с высоким разрешением). Тогда искомое значение будет равно

$$
\begin{equation}
  Z^l_F(\mathbf{x}) = Z^l_{F1}(\mathbf{x}) + Z^l_{F2}(\mathbf{x})
\end{equation}
\qquad \qquad (1) \qquad , где
$$

$$
\begin{equation}
  Z^l_{F1}(\mathbf{x}) - тренд; \qquad Z^l_{F2}(\mathbf{x}) - остаток.
\end{equation}
$$

Для нахождения первой части, то есть тренда, поступим следующим образом:

**1.** Возьмем канал с высоким разрешением и уменьшим его разрешение до каналов с грубым разрешением, применив для этого свертку с ядром в соответствии с функцией распределения ([Point Spread Function](https://en.wikipedia.org/wiki/Point_spread_function), PSF) и отмасштабировав изображение. Эта функция является результатом такого физического явления как дисперсия, что неизбежно возникает в небольших оптических системах и приводит к тому, что бесконечно удаленная точка не изображается в виде точки, а представляет из себя некоторое расплывчатое пятно. На этом шаге получим значения отмасштабированного свернутого канала с уже низким разрешением:

$$
\begin{equation}
  Z_C = h_C (\mathbf{x}) \ast Z_F (\mathbf{x}) = \int h_C (\mathbf{x} - \mathbf{y}) Z_F (\mathbf{y}) \mathrm{d} \mathbf{y}  
\end{equation}
\qquad \qquad (2) \qquad , где
$$

$$
\begin{equation}
  h_C (\mathbf{x}) - Point \ Spread \ Function; \qquad \ast - операция \ свертки;   
\end{equation}
$$

**2.** Полученный на предыдущем этапе результат используем для решения методом наименьших квадратов уравнения:

$$
\begin{equation}
  Z_C^l = a_l Z_C (\mathbf{x}) + b_l
\end{equation}
\qquad \qquad (3)
$$

**3.** Получим тренд для итогового результата:

$$
\begin{equation}
  Z^l_{F1}(\mathbf{x_0}) = a_l Z_F (\mathbf{x_0}) + b_l
\end{equation}
\qquad \qquad (4)
$$

**4.** Получим остаток:

$$
\begin{equation}
  Z^l_{С2}(\mathbf{x}) = R (\mathbf{x}) = Z^l_C (\mathbf{x}) - [a_l Z_C (\mathbf{x}) + b_l]
\end{equation}
\qquad \qquad (5)
$$

**5.** Построим экспериментальные семивариограммы и подберем наилучшую теоретическую модель $\gamma^l_{FF} (\mathbf{s})$. 

**6.** Найдем следующие выражения:

$$
\begin{equation}
  \begin{matrix}
    \gamma^l_{FC} (\mathbf{x}_0, \mathbf{x}_j) = \frac{1}{\sigma} \sum_{m = 1}^{\sigma} \gamma^l_{FF} (\mathbf{s}_m) \\
    \gamma^l_{CC} (\mathbf{x}_i, \mathbf{x}_j) = \frac{1}{\sigma^2} \sum_{m = 1}^{\sigma} \sum_{m' = 1}^{\sigma} \gamma^l_{FF} (\mathbf{s}_{mm'})
 \end{matrix}
\end{equation}
\qquad \qquad (6)
$$

**7.** Найденные значения используются для построения системы уравнений Кригинга:

$$
\begin{equation}
  \begin{bmatrix}
    \gamma^l_{CC} (\mathbf{x}_1, \mathbf{x}_1) &  \cdots &  \gamma^l_{CC} (\mathbf{x}_1, \mathbf{x}_N) & 1 \\ 
    \vdots & \ddots & \vdots & \vdots \\ 
    \gamma^l_{CC} (\mathbf{x}_N, \mathbf{x}_1) & \cdots & \gamma^l_{CC} (\mathbf{x}_N, \mathbf{x}_N) & 1 \\ 
    1 & \cdots & 1 & 0
  \end{bmatrix}
  \begin{bmatrix}
    \lambda_1 \\ 
    \vdots \\ 
    \lambda_N \\ 
    \vartheta
  \end{bmatrix}
  =
  \begin{bmatrix}
    \gamma^l_{FC} (\mathbf{x}_0, \mathbf{x}_1) \\ 
    \vdots \\ 
    \gamma^l_{FC} (\mathbf{x}_0, \mathbf{x}_N) \\ 
    1
  \end{bmatrix}
\end{equation}
\qquad \qquad (7)
$$

**8.** Решив систему, найдем веса, которые используются для улучшения разрешения остатка:

$$
\begin{equation}
  Z^l_{F2}(\mathbf{x_0}) = \sum_{i = 1}^{N} \lambda_i Z^l_{C2}(\mathbf{x}_i)
\end{equation}
\qquad \qquad (8)
$$

## Functions

### pansharpening

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Use some fine images as covariative for sharpening coarse images. There are a lot of setting parametrs which can be missed if you want calculation will be done automatically. Note that some of them require some information about semivariogram analysis to be correct. If you do not know anything about it, it would be better miss all additional parametrs.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">fineImage</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list contains images with fine resolution. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coarseImage</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list contains images with coarse resolution. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">geometry</td>
    <td class="tg-c3ow">{geometry}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Geometry object in which bounds all calculation will be done.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">geometry = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that polygon will conclude all image).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">model</td>
    <td class="tg-c3ow">{String}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The type of using model. You can choose: </span><span style="font-weight:bold;font-style:italic">'power'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'spher'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">. Use it if you know; in another case miss it.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
  <tr>
    <td class="tg-7btt">iterate</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of repeating iterate process. Use it if you want to pick the model. In another case miss it.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">iterate = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that it will be 50 or null depending on model type).</span><br></td>
  </tr>
  <tr>
    <td class="tg-7btt">coeff</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of coefficients for using model. Note that you should use it correctly or miss.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that these coefficients will be calculated automatically).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">PSF</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Point spread function (PSF) which using in convolution process.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">PSF = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(the box 3 by 3; all elements contain weight that equal [1 / coarse resolution]).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of sliding window using for finding fine residual. Note that this size shoul be choosen depending on semivariogram function. This number shoul be equal the range of graphic. If you do not know correct value, miss it.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">windowSize = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that size will be calculated automatically).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">initValues</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of init values for iterating process. Note that you should use it only for some types of model. If you do not know anything about it, miss it.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">initValues = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that init values will be calculated automatically if it is required).</span></td>
  </tr>
</table>

##### Return: image with fine sharpening bands.
#### Return: image

***

### sizeKernel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate size kernel for convolution process using coarse and fine resolutions.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Coarse resolution of using image. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">fineResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Fine resolution of using image. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: size for using kernel.
#### Return: number

***

### psf

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Construct kernel for convolution process.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">sizeKernel</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of using kernel for convolution process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">typeKernel</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list for constructing kernel. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">typeKernel = 'auto'</span><span style="font-style:italic;color:rgb(254, 0, 0)"> </span><span style="font-style:italic;text-decoration:underline;color:rgb(0, 0, 0)">(the box 3 by 3; all elements are equal [1 / sizeKernel^2]).</span></td>
  </tr>
</table>

##### Return: kernel for convolution process.
#### Return: kernel

***

### convolution

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Convolve giving image using your kernel.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The image using in convolution process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">kernel</td>
    <td class="tg-c3ow">{kernel}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The kernel using for convolution process</span>. <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: convolved image.
#### Return: image

***

### downscaling

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Set new resolution for using image.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The image using in downscaling process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">proj</td>
    <td class="tg-c3ow">{projection}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The projection using for downscaling process</span>. <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">sizeKernel</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The scale factor of result image resolution. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: downscaled image.
#### Return: image

***

### getLinRegression

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for linear regression model using two images as independent and dependent variables.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">downscaleImage</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Independent images using in linear regression model. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coarseImage</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Dependent image using in linear regression model. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">scale</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The scale factor of result image resolution. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">scale = 'auto'</span><span style="font-style:italic"> </span><span style="font-style:italic;text-decoration:underline">(this means that scale of coarse image will be used).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">proj</td>
    <td class="tg-c3ow">{projection}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The projection of result image. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">proj = 'auto'</span><span style="font-style:italic;color:rgb(254, 0, 0)"> </span><span style="font-style:italic;text-decoration:underline;color:rgb(0, 0, 0)">(this means that projection of coarse image will be used).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">geometry</td>
    <td class="tg-c3ow">{geometry}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Geometry object in which bounds linear regression will be calculated. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">geometry = 'auto'</span><span style="font-style:italic;color:rgb(254, 0, 0)"> </span><span style="font-style:italic;text-decoration:underline;color:rgb(0, 0, 0)">(this means that polygon will contains all coarse image).</span></td>
  </tr>
</table>

##### Return: list of coefficients for linear regression model.
#### Return: list

***

### trend

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get trend for the image using another image and linear regression coefficients.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Images which will be used for finding trend with linear regression coefficients. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coeff</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Linear regression coefficients . </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: trend for using image.
#### Return: image

***

### residual

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get residual for the image using coarse image and trend.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Image which will be used for finding residual. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">trend</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Trend of giving image . </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: residual for using image.
#### Return: image

***

### distance

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate Euclidean distance between 2 points which coordinates are given in Cortesian Coordinate system such {x,y}.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">x1</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">x-coordinate of first point. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">y1</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">y-coordinate of first point . </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">x2</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">x-coordinate of second point. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">y2</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">y-coordinate of second point . </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: Euclidean distance between 2 points.
#### Return: number

***

### CtoCsemivariogram

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate coarse-to-coarse semivariogram value for 2 pixels.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">Ipixel</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of i-pixel in sliding window. For example, if this pixel places in second column and third row and window size is 3, the number will be equal 8. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">Jpixel</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of j-pixel in sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Distance between two nearest pixels that do not lie in the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic"> The size of sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">sizeRatio</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The count of fine pixels within a coarse pixel. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of coefficients for using semivariogram model. Note that this amount should be correct and depending on using model. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span><span style="font-style:italic;color:rgb(254, 0, 0)"> </span><span style="font-style:italic;text-decoration:underline;color:rgb(0, 0, 0)">(this means that all coefficients will be equal 1).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">Using semivariogram module. See Semivariogram module to get some details.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
</table>

##### Return: coarse-to-coarse semivariogram value.
#### Return: number

***

### FtoCsemivariogram

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate fine-to-coarse semivariogram value for central pixel of sliding window and another pixel within it.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">pixelCenter</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of center pixel in sliding window. For example, if window size is 3, the number will be equal 5. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">Jpixel</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of j-pixel in sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Distance between two nearest pixels that do not lie in the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic"> The size of sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">sizeRatio</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The count of fine pixels within a coarse pixel. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of coefficients for using semivariogram model. Note that this amount should be correct and depending on using model. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span><span style="font-style:italic;color:rgb(254, 0, 0)"> </span><span style="font-style:italic;text-decoration:underline;color:rgb(0, 0, 0)">(this means that all coefficients will be equal 1).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">Using semivariogram module. See Semivariogram module to get some details.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
</table>

##### Return: fine-to-coarse semivariogram value.
#### Return: number

***

### krigingMatrix

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate kriging matrix which contains all combinations of coarse-to-coarse semivariogram values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Distance between two nearest pixels that do not lie in the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">sizeRatio</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The count of fine pixels within a coarse pixel.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coeff</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of coefficients for using semivariogram model. Note that this amount should be correct and depending on using model.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that all coefficients will be equal 1).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">Using semivariogram module. See Semivariogram module to get some details.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
</table>

##### Return: matrix for Kriging equations.
#### Return: list

***

### krigingVector

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate kriging vector which contains all combinations of fine-to-coarse semivariogram values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Distance between two nearest pixels that do not lie in the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">sizeRatio</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The count of fine pixels within a coarse pixel.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coeff</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of coefficients for using semivariogram model. Note that this amount should be correct and depending on using model.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that all coefficients will be equal 1).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">Using semivariogram module. See Semivariogram module to get some details.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
</table>

##### Return: vector for Kriging equations.
#### Return: list

***

### getWeights

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate table of weights as solving kriging equations.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">coarseResolution</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Distance between two nearest pixels that do not lie in the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">windowSize</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The size of sliding window. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">sizeRatio</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The count of fine pixels within a coarse pixel.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">coeff</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of coefficients for using semivariogram model. Note that this amount should be correct and depending on using model.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that all coefficients will be equal 1).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">Using semivariogram module. See Semivariogram module to get some details.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
</table>

##### Return: weight matrix for sliding window.
#### Return: list

***

## Script

In [ ]:
var Semivariogram = require('users/vasilevnikolay18/Images_fusion:Semivariogram');

// Pansharpening: sharp selected bands:

// fineImage {list} - images with fine resolution;
// coarseImage {list} - images with coarse resolution;
// iterate {number}  - counts of repeating iterating process {default: 50};
// coeff {object} - model's coefficients for constructing semivariogram (default: all 1);
// model {String} - the name of using model (default: 'powExp');
// PSF {list} - initial values for iterating process {default: auto};
// geometry {geometry} - geometry object in which the image locates {default: all image};
// windowSize {default: auto}
// initValues {list} - initial values for iterating process (default: 'auto');

// return: sharpening image; {image};

exports.pansharpening = function (fineImage, coarseImage, geometry, model, iterate, coeff, PSF, windowSize, initValues) {
  
  fineImage = ee.Image(fineImage);
  var coarseBands = ee.Image(coarseImage).bandNames();
  
  var first_channel = ee.String(fineImage.bandNames().get(0));
  var fineResolution = fineImage.select(first_channel).projection().nominalScale();
  var proj = fineImage.select(first_channel).projection();  
  geometry = ee.Algorithms.If(ee.Algorithms.IsEqual(geometry, null), ee.Feature(fineImage).geometry(), geometry);
  
  var pansharp_alltogether = coarseBands.map(function (coarseBand) {
  
    var coarseImage__ = coarseImage.select(ee.String(coarseBand));
  
    var coarseResolution = coarseImage__.projection().nominalScale();
  
    var Size_kernel = sizeKernel(coarseResolution, fineResolution);
   
    var Point_spread_func = psf(Size_kernel, PSF);
  
    var FineImage_convolved = convolution(fineImage, Point_spread_func);
    
    var FineImage_downscaled = downscaling(FineImage_convolved, proj, Size_kernel);
    
    var Linear_regression = getLinRegression(FineImage_downscaled, coarseImage__, coarseResolution, proj, geometry);
  
    var Trend_result = trend(fineImage, Linear_regression);
  
    var Trend_coarse = trend(FineImage_downscaled, Linear_regression);
  
    var Coarse_residual = residual(coarseImage__, Trend_coarse);
  
    var sizeRatio = ee.Number(Size_kernel).pow(2);
   
    var Experiment_data = ee.Algorithms.If((ee.Algorithms.IsEqual(coeff, null)) || (ee.Algorithms.IsEqual(windowSize, null)), 
                          Semivariogram.semivariogramExperiment(Coarse_residual, 0, 15, geometry, coarseResolution, proj, 'Smooth'), null);
    
    coeff = ee.Algorithms.If(ee.Algorithms.IsEqual(coeff, null),
            Semivariogram.getCoeff(Experiment_data, model, iterate, initValues), coeff);
    
    windowSize = ee.Algorithms.If(ee.Algorithms.IsEqual(windowSize, null), 
                 ee.Number(ee.List(Semivariogram.getParameters(Experiment_data)).get(1))
                 .multiply(2).divide(coarseResolution).round(), 5);
    
    windowSize = ee.Algorithms.If(ee.Number(windowSize).mod(2).eq(0), ee.Number(windowSize).add(1), windowSize);
    
    var weights = getWeights(coarseResolution, sizeRatio, windowSize, coeff, model);
    
    var Kernel_for_residual = psf(ee.List(weights).length(), weights);
    
    var residual_convolved = convolution(Coarse_residual, Kernel_for_residual);
    
    var residual_downscaling = downscaling(residual_convolved, proj, 1);
    
    var fine_image = ee.Image(Trend_result).add(ee.Image(residual_downscaling));
  
    return ee.Image(fine_image);
    
  });
  
  return ee.Image(pansharp_alltogether);
  
};


// sizeKernel: set size for using kernel:

// coarseResolution {number} - coarse resolution of using band;
// fineResolution {number} - fine resolution of using band;

// return: size; {number};

var sizeKernel = function(coarseResolution, fineResolution) {
  
  coarseResolution = ee.Number(coarseResolution);
  fineResolution = ee.Number(fineResolution);
  
  return coarseResolution.divide(fineResolution);
  
};


// psf: set a Point Spread Function (PSF):

// sizeKernel {number} - size of kernel using for psf;
// typeKernel {number} - type of kernel using for psf {default: 'auto'};

// return: kernel; {kernel};

var psf = function(sizeKernel, typeKernel) {
  
  sizeKernel = ee.Number(sizeKernel);
  
  typeKernel = ee.Algorithms.If(ee.Algorithms.IsEqual(typeKernel, null), 
               ee.List.repeat(ee.List.repeat(ee.Number(1).divide(sizeKernel.pow(2)), sizeKernel), sizeKernel), 
               typeKernel);
  
  var kernel = ee.Kernel.fixed({
    width: sizeKernel,
    height: sizeKernel, 
    weights: typeKernel,
    x: sizeKernel.divide(2).round().subtract(-1).multiply(-1),
    y: sizeKernel.divide(2).round().subtract(-1).multiply(-1), 
    normalize: false
  });
  
  return kernel;
  
}; 


// convolution: convolution operation:

// image {image} - image for convolution;
// kernel {kernel} - kernel for convolution;

// return convolved image; {image};

var convolution = function(image, kernel) {
  
  return ee.Image(image).convolve(kernel);
  
};


// downscaling: dowscaling images;

// image {image} - list of images;
// proj {projection} - the project of the image;
// sizeKernel {number} - size of kernel using for psf;

// return downscaling image with bands; {image}

var downscaling = function(image, proj, sizeKernel) {
  
  image = ee.Image(image);
  proj = ee.Projection(proj);
  
  return image.reduceResolution({
           reducer: ee.Reducer.mean(),
           bestEffort: true
         })
         .reproject(proj.scale(sizeKernel, sizeKernel));
  
};


// getLinRegression: get coefficients using linear regression: y = a*x + b:

// downscaleImage {image} - independent image;
// coarseImage {image} - dependent image;
// scale {number} - scale of image {default: nominal scale};
// proj {projection} - the project of the image (default: projection of giving image);
// geometry {geometry} - geometry object in which the image locates {default: coarseImage};

// return: list of coefficients [a, b]; {list};

var getLinRegression = function(downscaleImage, coarseImage, scale, proj, geometry) {
  
  geometry = ee.Algorithms.If(ee.Algorithms.IsEqual(geometry, null), ee.Feature(coarseImage).geometry(), geometry);
  scale = ee.Algorithms.If(ee.Algorithms.IsEqual(scale, null), coarseImage.projection().nominalScale(), scale);
  proj = ee.Algorithms.If(ee.Algorithms.IsEqual(proj, null), coarseImage.projection(), proj);
  
  var ones = ee.Image(1);
  
  var linearFitter = ee.Reducer.linearRegression({
    numX: ee.Number(downscaleImage.bandNames().length()).add(1),
    numY: 1
  });
  
  var matrix = ee.Image.cat([downscaleImage, ones, coarseImage]);
  
  var coefficients = matrix.reduceRegion({
    reducer: linearFitter,
    geometry: geometry,
    scale: scale,
    crs: proj,
    bestEffort: true
  });
  
  var coeff = ee.Array(coefficients.get('coefficients')).toList();
  
  return coeff;
  
};


// trend: get trend image in fine resolution:

// image {image} - image using for getting trend;
// coeff {object} - linear regression coefficients;

// return trend; {image};

var trend = function(image, coeff) {
  
  image = ee.Image(image);
  coeff = ee.List(coeff);
  
  var b = coeff.get(-1);
  var a = coeff.remove(b).flatten();
  b = ee.List(b).get(0);
  
  return image.multiply(ee.Image.constant(a)).reduce(ee.Reducer.sum().unweighted()).add(ee.Image.constant(b));
  
};


// residual: finding residual:

// image {image} - coarse image using for getting residual;
// trend {image} - trend;

// return residual; {image};

var residual = function (image, trend) {
  
  return ee.Image(image).subtract(ee.Image(trend));

};


// distance: get distance between two points using Cartesian coordinate system;

// point1 {object} - coordinates of first point;
// point2 {object} - coordinates of second point;

// return: distance; {number};

var distance = function (x1, y1, x2, y2) {
  
  x1 = ee.Number(x1);
  y1 = ee.Number(y1);
  x2 = ee.Number(x2);
  y2 = ee.Number(y2);
  
  var delta_x = x1.subtract(x2);
  var delta_y = y1.subtract(y2);
  
  return delta_x.pow(2).add(delta_y.pow(2)).sqrt();
  
};


// CtoCsemivariogram: get coarse-to-coarse semivariogram;

// Ipixel {number} - number of i pixel in the window;
// Jpixel {number} - number of j pixel in the window;
// coarseResolution {number} - pixel size in the window;
// windowSize {number} - size of the window;
// sizeRatio {number} - fine pixel size ratio;
// semivarFormula {function} - formula for calculating semivariogram;
// coeff {object} - model's coefficients for constructing semivariogram {default: 'auto'};
// model {string} - the name of using model {default: 'powExp'};

// return: coarse-to-coarse semivariogram; {number};

var CtoCsemivariogram = function (Ipixel, Jpixel, coarseResolution, windowSize, sizeRatio, coeff, model) {
  
  Ipixel = ee.Number(Ipixel);
  Jpixel = ee.Number(Jpixel);
  coarseResolution = ee.Number(coarseResolution);
  windowSize = ee.Number(windowSize);
  sizeRatio = ee.Number(sizeRatio);
  
  var semivarFormula = function (lag) {
    return Semivariogram.semivariogramTheory(lag, model, coeff);
  };
  
  var Jpixel_col = ee.Number(ee.Algorithms.If(Jpixel.mod(windowSize).eq(0), 
               windowSize, Jpixel.mod(windowSize)));
  var Jpixel_row = Jpixel.subtract(Jpixel_col).divide(windowSize).add(1);
  
  var Ipixel_col = ee.Number(ee.Algorithms.If(Ipixel.mod(windowSize).eq(0), 
               windowSize, Ipixel.mod(windowSize)));
  var Ipixel_row = Ipixel.subtract(Ipixel_col).divide(windowSize).add(1);
  
  var Ipixel_coord_x = coarseResolution.multiply(Ipixel_col.subtract(1)).add(coarseResolution.divide(2));
  var Ipixel_coord_y = coarseResolution.multiply(Ipixel_row.subtract(1)).add(coarseResolution.divide(2));  
  
  var Jpixel_coord_x = coarseResolution.multiply(Jpixel_col.subtract(1)).add(coarseResolution.divide(2));
  var Jpixel_coord_y = coarseResolution.multiply(Jpixel_row.subtract(1)).add(coarseResolution.divide(2));  
    
  var list_inner = ee.List.sequence(1, sizeRatio);
  var list_outer = ee.List.sequence(1, sizeRatio);
  var subresolution = coarseResolution.divide(sizeRatio.sqrt());
  
  var sum_outer = list_outer.iterate(function (m, sum2){
    
    m = ee.Number(m);
    
    var Ipixel_fine_col = ee.Number(ee.Algorithms.If(ee.Number(m).mod(sizeRatio.sqrt()).eq(0), 
                          sizeRatio.sqrt(), ee.Number(m).mod(sizeRatio.sqrt())));
    var Ipixel_fine_row = ee.Number(m).subtract(Ipixel_fine_col).divide(sizeRatio.sqrt()).add(1);
      
    var Ipixel_fine_coord_x = Ipixel_coord_x.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Ipixel_fine_col.subtract(1))
                              .add(subresolution.divide(2)));
      
    var Ipixel_fine_coord_y = Ipixel_coord_y.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Ipixel_fine_row.subtract(1))
                              .add(subresolution.divide(2)));
  
      var sum_inner = list_inner.iterate(function (n, sum1){
        
        n = ee.Number(n);
        
        var Jpixel_fine_col = ee.Number(ee.Algorithms.If(ee.Number(n).mod(sizeRatio.sqrt()).eq(0), 
                          sizeRatio.sqrt(), ee.Number(n).mod(sizeRatio.sqrt())));
        var Jpixel_fine_row = ee.Number(n).subtract(Jpixel_fine_col).divide(sizeRatio.sqrt()).add(1);
      
        var Jpixel_fine_coord_x = Jpixel_coord_x.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Jpixel_fine_col.subtract(1))
                              .add(subresolution.divide(2)));
      
        var Jpixel_fine_coord_y = Jpixel_coord_y.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Jpixel_fine_row.subtract(1))
                              .add(subresolution.divide(2)));
        
        var lag = distance(Ipixel_fine_coord_x, Ipixel_fine_coord_y, Jpixel_fine_coord_x, Jpixel_fine_coord_y);
        
        lag = ee.Algorithms.If(ee.Algorithms.IsEqual(Ipixel, Jpixel), 
              ee.Number(lag).divide(2), lag);
        
        return ee.Number(sum1).add(ee.Number(semivarFormula(lag)));
        
      }, sum2);
      
      return sum_inner;
      
  }, 0);  
    
  return ee.Number(sum_outer).divide(sizeRatio);
    
};


// FtoCsemivariogram: get fine-to-coarse semivariogram;

// pixelCenter {number} - number of center pixel in the window;
// Jpixel {number} - number of j pixel in the window;
// coarseResolution {number} - pixel size in the window;
// windowSize {number} - size of the window;
// sizeRatio {number} - fine pixel size ratio;
// coeff {object} - model's coefficients for constructing semivariogram {default: 'auto'};
// model {string} - the name of using model {default: 'powExp'};

// return: coarse-to-coarse semivariogram; {number};

var FtoCsemivariogram = function (pixelCenter, Jpixel, coarseResolution, windowSize, sizeRatio, coeff, model) {
  
  pixelCenter = ee.Number(pixelCenter);
  Jpixel = ee.Number(Jpixel);
  coarseResolution = ee.Number(coarseResolution);
  windowSize = ee.Number(windowSize);
  sizeRatio = ee.Number(sizeRatio);
  
  var semivarFormula = function (lag) {
    return Semivariogram.semivariogramTheory(lag, model, coeff);
  };
  
  var Jpixel_col = ee.Number(ee.Algorithms.If(Jpixel.mod(windowSize).eq(0), 
                   windowSize, Jpixel.mod(windowSize)));
  var Jpixel_row = Jpixel.subtract(Jpixel_col).divide(windowSize).add(1);
  
  var pixelCenter_col = ee.Number(ee.Algorithms.If(pixelCenter.mod(windowSize).eq(0), 
                        windowSize, pixelCenter.mod(windowSize)));
  var pixelCenter_row = pixelCenter.subtract(pixelCenter_col).divide(windowSize).add(1);
  
  var pixelCenter_coord_x = coarseResolution.multiply(pixelCenter_col.subtract(1)).add(coarseResolution.divide(2));
  var pixelCenter_coord_y = coarseResolution.multiply(pixelCenter_row.subtract(1)).add(coarseResolution.divide(2));  
  
  var Jpixel_coord_x = coarseResolution.multiply(Jpixel_col.subtract(1)).add(coarseResolution.divide(2));
  var Jpixel_coord_y = coarseResolution.multiply(Jpixel_row.subtract(1)).add(coarseResolution.divide(2));  

  var list = ee.List.sequence(1, sizeRatio);
  var subresolution = coarseResolution.divide(sizeRatio.sqrt());

  var sum = list.iterate(function (n, sum1){
        
    n = ee.Number(n);
        
    var Jpixel_fine_col = ee.Number(ee.Algorithms.If(ee.Number(n).mod(sizeRatio.sqrt()).eq(0), 
                          sizeRatio.sqrt(), ee.Number(n).mod(sizeRatio.sqrt())));
    var Jpixel_fine_row = ee.Number(n).subtract(Jpixel_fine_col).divide(sizeRatio.sqrt()).add(1);
      
    var Jpixel_fine_coord_x = Jpixel_coord_x.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Jpixel_fine_col.subtract(1))
                              .add(subresolution.divide(2)));
      
    var Jpixel_fine_coord_y = Jpixel_coord_y.subtract(coarseResolution.divide(2))
                              .add(subresolution.multiply(Jpixel_fine_row.subtract(1))
                              .add(subresolution.divide(2)));
        
    var lag = distance(pixelCenter_coord_x, pixelCenter_coord_y, Jpixel_fine_coord_x, Jpixel_fine_coord_y);
        
    return ee.Number(sum1).add(ee.Number(semivarFormula(lag)));
        
    }, 0);
    
  return ee.Number(sum).divide(sizeRatio.sqrt());
  
};


// krigingMatrix: construct matrix using coarse-to-coarse semivariograms:

// coarseResolution {number} - pixel size in the window;
// windowSize {number} - size of the window;
// sizeRatio {number} - fine pixel size ratio;
// coeff {object} - model's coefficients for constructing semivariogram {default: 'auto'};
// model {string} - the name of using model {default: 'powExp'};

// return: 2-D array (matrix); {list[list1, list2...]};

var krigingMatrix = function (coarseResolution, windowSize, sizeRatio, coeff, model) {
    
  coarseResolution = ee.Number(coarseResolution);
  windowSize = ee.Number(windowSize);
  sizeRatio = ee.Number(sizeRatio);
    
  var matrix_size = windowSize.pow(2);  
  
  var list_row = ee.List.sequence(0, matrix_size.subtract(1));
  var list_col = ee.List.sequence(0, matrix_size.subtract(1));  
  var Matrix = ee.List.repeat(ee.List.repeat(1, matrix_size.add(1)), matrix_size.add(1));
  
  var cols = list_col.iterate(function (n, table1) {
    var rows = list_row.iterate(function (m, table2) {
      
      var pixel_i = ee.Number(m).add(1);
      var pixel_j = ee.Number(n).add(1);
      
      var element = CtoCsemivariogram(pixel_i, pixel_j, coarseResolution, windowSize, sizeRatio, coeff, model);
      
      return ee.List(table2).set(m, ee.List(ee.List(table2).get(m)).set(n, element));
        
    }, table1); 
    return rows;
  }, Matrix);
    
  var Kriging_matrix = cols;
  Kriging_matrix = ee.List(Kriging_matrix).set(-1, ee.List(ee.List(Kriging_matrix).get(-1)).set(-1, 0));
  
  return Kriging_matrix;
  
}; 


// krigingVector: construct matrix using coarse-to-coarse semivariograms:

// coarseResolution {number} - pixel size in the window;
// windowSize {number} - size of the window;
// sizeRatio {number} - fine pixel size ratio;
// coeff {object} - model's coefficients for constructing semivariogram {default: 'auto'};
// model {string} - the name of using model {default: 'powExp'};

// return: 1-D array (vector); {list};

var krigingVector = function (coarseResolution, windowSize, sizeRatio, coeff, model) {
    
  coarseResolution = ee.Number(coarseResolution);
  windowSize = ee.Number(windowSize);
  sizeRatio = ee.Number(sizeRatio);
    
  var matrix_size = windowSize.pow(2); 
  var list = ee.List.sequence(0, matrix_size.subtract(1));
  var matrix = ee.List.repeat(ee.List.repeat(0, matrix_size.add(1)), 2);
  
  var pixel_0 = matrix_size.divide(2).round();
  
  var col = list.iterate(function (m, col0) {
    
    var pixel_j = ee.Number(m).add(1);
    
    var element = FtoCsemivariogram(pixel_0, pixel_j, coarseResolution, windowSize, sizeRatio, coeff, model);
       
    return ee.List(col0).set(0, ee.List(ee.List(col0).get(0)).set(m, element));
    
  }, matrix);  

  var Kriging_vector = col;
  Kriging_vector = ee.List(Kriging_vector).set(0, ee.List(ee.List(Kriging_vector).get(0)).set(-1, 1));
  Kriging_vector = ee.Array(Kriging_vector).matrixTranspose(0, 1).toList();
    
  return Kriging_vector;
  
};


// getWeights: calculate weights using kriging equations;

// coarseResolution {number} - pixel size in the window;
// sizeRatio {number} - fine pixel size ratio;
// windowSize {number} - size of the window {default: 'auto'};
// coeff {object} - model's coefficients for constructing semivariogram (default: 'auto');
// model {string} - the name of using model (default: 'powExp');

// return: 2-D array of weights (matrix); {list[list1, list2,...]};

var getWeights = function (coarseResolution, sizeRatio, windowSize, coeff, model) {
  
  coarseResolution = ee.Number(coarseResolution);
  windowSize = ee.Number(windowSize);
  sizeRatio = ee.Number(sizeRatio);
  
  var Kriging_matrix = ee.Array(krigingMatrix(coarseResolution, windowSize, sizeRatio, coeff, model));
  var Kriging_vector = ee.Array(krigingVector(coarseResolution, windowSize, sizeRatio, coeff, model));
  
  var Weights = Kriging_matrix.matrixSolve(Kriging_vector).matrixTranspose(0, 1).toList().get(0);
  Weights = ee.List(Weights).remove(ee.List(Weights).get(-1));
  
  var Table = ee.List.repeat(ee.List.repeat(0, windowSize), windowSize);
  var list = ee.List.sequence(0, windowSize.subtract(1));
  var Table_weights = list.iterate(function (row, Table0) {
    row = ee.Number(row);
    var segment = Weights.slice(row.multiply(windowSize), row.add(1).multiply(windowSize));
    return ee.List(Table0).set(row, segment);
  }, Table);
  
  return Table_weights;
  
};